In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.chdir('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import sys
sys.path.append("Marigold")

import torch
import numpy as np
import jhutil; jhutil.color_log(1111, )

 1111 


In [2]:
import sys
sys.argv = [
    "ludvig_uplift.py", 
    "--colmap_dir", "./dataset/llff_data/fern/", 
    "--gs_source", "./dataset/llff_data/fern/gs/point_cloud/iteration_30000/point_cloud.ply", 
    "--config", "configs/dif_NVOS.yaml", 
    "--height", "1199", 
    "--width", "1600", 
    "--tag", "fern"
]

In [3]:
from ludvig_uplift import *

args = parse_args()
reproducibility(0)
model = LUDVIGUplift(args)

Reading camera 20/20
735471 Gaussians.


# 1. Load dataset

In [4]:
t0 = time()
print("Uplifting features...")
directory = model.config['feature'].pop(
    'directory',
    os.path.join(model.colmap_dir, 'images')
)
dataset = config_to_instance(
    directory=directory,
    gaussian=model.gaussian,
    cameras=model.colmap_cameras,
    render_fn=model.render,
    scene=model.scene,
    height=model.img_height,
    width=model.img_width,
    **model.config.feature,
)
loader = iter(dataset)

Uplifting features...
   cccc  "cache file found, skipping dino_extract"


# 2. Uplift

In [5]:
features, _ = uplifting(
    loader,
    model.gaussian,
    prune_gaussians=model.config.get("prune_gaussians", None),
)
if model.config.get('normalize', False):
    print("l2-normalizing uplifted features.")
    features /= features.norm(dim=1, keepdim=True) + 1e-6
print(
    f"Total time for preprocessing + uplifting {len(model.colmap_cameras)} images: {round(time()-t0)}s"
)
model.features = features

20it [00:03,  6.22it/s]


Time for uplifting 20 views: 3.2s
Keeping 367735 points out of 735471.
Total time for preprocessing + uplifting 20 images: 6s


# 3. Evaluate

In [ ]:
########################################################
# load eval_fn
########################################################

from evaluation.spin_nvos.diffusion import SegmentationDiffusionNVOS

os.makedirs(model.logdir, exist_ok=True)
cfg_path = os.path.join(model.logdir, "config.yaml")
yaml.dump(model.config, open(cfg_path, "w"))
eval_kwargs = model.config.get("evaluation", dict())

eval_fn: SegmentationDiffusionNVOS = config_to_instance(
    gaussian=model.gaussian,
    features=model.features,
    render_fn=model.render,
    render_rgb=model.render_rgb,
    logdir=model.logdir,
    image_dir=model.colmap_dir,
    colmap_cameras=model.colmap_cameras,
    scene=model.scene,
    height=model.img_height,
    width=model.img_width,
    **model.config.evaluation,
)

_IncompatibleKeys(missing_keys=['mask_decoder.hf_token.weight', 'mask_decoder.hf_mlp.layers.0.weight', 'mask_decoder.hf_mlp.layers.0.bias', 'mask_decoder.hf_mlp.layers.1.weight', 'mask_decoder.hf_mlp.layers.1.bias', 'mask_decoder.hf_mlp.layers.2.weight', 'mask_decoder.hf_mlp.layers.2.bias', 'mask_decoder.compress_vit_feat.0.weight', 'mask_decoder.compress_vit_feat.0.bias', 'mask_decoder.compress_vit_feat.1.weight', 'mask_decoder.compress_vit_feat.1.bias', 'mask_decoder.compress_vit_feat.3.weight', 'mask_decoder.compress_vit_feat.3.bias', 'mask_decoder.embedding_encoder.0.weight', 'mask_decoder.embedding_encoder.0.bias', 'mask_decoder.embedding_encoder.1.weight', 'mask_decoder.embedding_encoder.1.bias', 'mask_decoder.embedding_encoder.3.weight', 'mask_decoder.embedding_encoder.3.bias', 'mask_decoder.embedding_maskfeature.0.weight', 'mask_decoder.embedding_maskfeature.0.bias', 'mask_decoder.embedding_maskfeature.1.weight', 'mask_decoder.embedding_maskfeature.1.bias', 'mask_decoder.embedd

In [11]:
args = eval_fn.hyperparameter_search()

6/6 | Bandwidths: 4.0, 16.0  IoU: 0.857 --  Best IoU: 0.891  Best bandwidths: 2.0, 16.0


In [12]:
eval_fn.evaluate(*args)

Feature and regularization bandwiths, chosen based on IoU with SAM mask: 2.0 16.0
IoU: 0.843
